In [1]:
import tensorflow as tf

# Checks for available GPUs and lists them
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Set TensorFlow to use only the first GPU
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 1 Logical GPU


2024-01-06 22:10:57.415897: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-06 22:10:57.415915: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-06 22:10:57.415919: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-06 22:10:57.415944: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-06 22:10:57.415959: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0


In [3]:
# Reshape the data to include a channel dimension
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))


In [4]:
#Get dimentions of data
_, a, b, c = x_train.shape

In [5]:
# Convert the data to a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Define a function for data augmentation
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

# Apply data augmentation to the training dataset
train_dataset = train_dataset.map(augment)


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(a,b,c)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [7]:
model.fit(train_dataset.batch(32), epochs=5)
model.evaluate(x_test, y_test)


Epoch 1/5
   1/1875 [..............................] - ETA: 8:15 - loss: 2.7479 - accuracy: 0.0625

2024-01-06 22:10:58.469514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 9s 5ms/step - loss: 0.6310 - accuracy: 0.8061
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5767 - accuracy: 0.8266
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5904 - accuracy: 0.8244
Epoch 4/5
1875/1875 [==============================] - 8s 5ms/step - loss: 0.5999 - accuracy: 0.8222
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.6608 - accuracy: 0.8055


[0.6607926487922668, 0.8054999709129333]